<a href="https://colab.research.google.com/github/Winest-Nigeria/fgan-I-298-R1/blob/main/2023/Triplet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.1 MB/s eta 0:00:00


In [ ]:
import PyPDF2 as pdf
import string

In [ ]:
# Path to your PDF file
pdf_file_path = '/content/sample_data/test_usecase_document.pdf'

In [ ]:
# Open the PDF file
with open(pdf_file_path, 'rb') as pdf_file:
  # Create a PDF reader object
  pdf_reader = pdf.PdfReader(pdf_file)

  # Initialize an empty string to store the extracted text
  pdf_text = ''

  # Iterate through each page and extract text
  for page_num in range(len(pdf_reader.pages)):
      page = pdf_reader.pages[page_num]
      pdf_text += page.extract_text()


In [ ]:
pdf_text[:100]

' \n  \n7.1  Import and export of knowledge in an autonomous network  \n  \nUse case ID  AN-usecase -001 '

In [ ]:
# Remove all puntuation in text
text= ''.join([c for c in pdf_text if c not in string.punctuation])

In [ ]:
text[:100]

' \n  \n71  Import and export of knowledge in an autonomous network  \n  \nUse case ID  ANusecase 001  \nU'

In [ ]:
len(text)

9655

In [ ]:

#Tokenize the pdf text
tokens = text.split()

In [ ]:

#show first 10 elements of token list
tokens[:10]

['71',
 'Import',
 'and',
 'export',
 'of',
 'knowledge',
 'in',
 'an',
 'autonomous',
 'network']

In [ ]:

#number of elements in tokens list
len(tokens)

1474

In [ ]:

#import nltk library and download necessary files
import nltk
from nltk import word_tokenize, pos_tag
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

In [ ]:

#Import nltk library stopwords
from nltk.corpus import stopwords

In [ ]:

#Remove stopwords from tokens
tokens=[word for word in tokens if word.lower() not in stopwords.words('english')]

In [ ]:

#Number of elements in new tokens list
len(tokens)

945

In [ ]:

#remove numbers and single characters word from tokens list
tokens = [word for word in tokens if not (word.isdigit() or len(word) == 1)]

In [ ]:
tokens = [word for word in tokens if not (word == 'Use' or word == 'use' or word == 'case')]

In [ ]:
len(tokens)

793

In [ ]:
tokens[:10]

['Import',
 'export',
 'knowledge',
 'autonomous',
 'network',
 'ID',
 'ANusecase',
 'description',
 'satisfy',
 'key']

In [ ]:

#import spacy library
import spacy
nlp = spacy.load("en_core_web_sm")

In [ ]:
word_list = tokens
# Join the words into a text string
text = ' '.join(word_list)

# Process the text with spaCy
doc = nlp(text)

# Extract nouns
noun = [token.text for token in doc if token.pos_ == "NOUN"]

# Print the extracted nouns
print(noun[:100])

['Import', 'export', 'knowledge', 'network', 'description', 'concepts', 'ANs', 'evolution', 'experimentation', 'adaptation', 'intervention', 'knowledge', 'knowledge', 'representation', 'data', 'environment', 'system', 'actions', 'configuration', 'options', 'measurement', 'parameters', 'elements', 'logic', 'scenarios', 'knowledge', 'actors', 'scenarios', 'steps', 'peer', 'entities', 'components', 'eg', 'evolution', 'exploration', 'configuration', 'automation', 'loops', 'consumption', 'components', 'components', 'peer', 'entities', 'category', 'scenario', 'behaviour', 'requirements', 'requirements', 'exchange', 'knowledge', 'components', 'ANUC01', 'optimization', 'knowledge', 'bases', 'optimizations', 'knowledge', 'bases', 'access', 'policies', 'arity', 'storage', 'interconnection', 'knowledge', 'bases', 'relation', 'problems', 'solutions', 'knowledge', 'ANUC01', 'creation', 'knowledge', 'bases', 'ion', 'humans', 'machines', 'contents', 'statistics', 'knowledge', 'components', 'network',

In [ ]:
len(noun)

398

In [ ]:

#using nltk go get the various part of speech
parts_of_speech = pos_tag(noun)
len(parts_of_speech)

398

In [ ]:
parts_of_speech[:10]

[('Import', 'NNP'),
 ('export', 'NN'),
 ('knowledge', 'NN'),
 ('network', 'NN'),
 ('description', 'NN'),
 ('concepts', 'NNS'),
 ('ANs', 'NNP'),
 ('evolution', 'NN'),
 ('experimentation', 'NN'),
 ('adaptation', 'NN')]

In [ ]:

#extract nouns from the tokens list
nouns = list(set(filter(lambda x: x[1]=="NN" , parts_of_speech)))

In [ ]:
nouns

In [ ]:
len(nouns)

74